In [ ]:
%%time
# Wir nutzen RAPIDS, um die Modelle GPU-beschleunigt trainieren zu können
import sys
!cp ../input/rapids/rapids.0.18.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True) # falls plotly mal nichts anzeigt, diese Zelle wieder ausführen

# Machine Learning für Zeitreihen
Dieses Notebook zeigt, wie klassische Machine-Learning-Algorithmen für das Forecasting von Zeitreihen genutzt werden können.

## Beispiel-Daten einlesen

Für unsere Beispiele nutzen wir einen Datensatz von der Johns Hopkins Universität mit täglich aktualisierten Zahlen zu den gemeldeten Infektionen mit COVID-19 aus verschiedenen Ländern.

In [ ]:
df = pd.read_csv(
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',
    dtype={'Country/Region':'category'}
)

df = df.loc[df['Country/Region'].isin(['France', 'Portugal', 'Spain'])] # wir nutzen nur die Daten für Deutschland, Italien, Spanien und Frankreich
df = df.loc[df['Province/State'].isnull()]
df = df.drop(columns=['Province/State', 'Lat', 'Long']) # die Spalten brauchen wir nicht
df = df.melt(id_vars='Country/Region', var_name='date', value_name='value_total') # vom Wide-Format in das Long-Format transformieren
df['date'] = pd.to_datetime(df['date'])
df.sort_values('date', inplace=True) # nach Datum sortieren
df.sample(5)

## Basis-Features für das Zeitreihen-Modell
Für unsere ersten Modelle nutzen wir 4 einfache Features:

- `value_daily` der tagesaktuelle Wert der neu gemeldeten Infektionen
- `value_trend` der Durchschnittswert der letzten 7-Tage
- `value_trend_slope` die Differenz zwischen dem aktuellen Trendwert und dem Trendwert vor 7 Tagen, was ungefähr der Steigung der Trendkurve entspricht
- `weekday` der aktuelle Wochentag

In [ ]:
df['value_daily'] = df.groupby('Country/Region')['value_total'].diff().fillna(0) # value_total sind die kumulierten Zahlen
df.loc[df['value_daily'] < 0, 'value_daily'] = 0
df.drop(columns='value_total', inplace=True)

df['value_trend'] = df.groupby('Country/Region')['value_daily'].rolling(window=7).mean().reset_index(level=0, drop=True)

df['value_trend_slope'] = df.groupby('Country/Region')['value_trend'].diff(7)

df['weekday'] = df['date'].dt.weekday

df.sample(5)

Darstellung der Werte und des Trends mit Plotly...

In [ ]:
df.groupby('Country/Region', observed=True)['value_daily'].mean()

In [ ]:
fig = px.line(df,
        x='date', y=['value_daily', 'value_trend'],
        title='Täglich gemeldete Neuinfektionen mit COVID-19',
        facet_row='Country/Region',
        labels=dict(date='Datum', value=''))
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1])) # 'Italy' anstatt 'Country/Region=Italy' für die Beschriftung der Subplots
fig.update_yaxes(matches=None) # individuelle y-Achsen-Skala je Diagramm
fig.show()

## ML-Modell trainieren: mehrere direkte Modelle (eins je Horizont)

Wir nehmen an, dass wir die neugemeldeten Infektionen für die nächsten 14 Tage vorhersagen möchten. Wir trainieren dafür 14 Modelle - eins für jeden Vorhersagehorizont. Die Features sind für jedes Modell identisch, aber jeweils mit einem weiter verschobenen Zielwert. Wir benutzen den Random-Forest für Regressionen aus RAPIDS cuML (NVIDIA) als Algorithmus.

In [ ]:
%%time

from cuml.ensemble import RandomForestRegressor

horizon = 14

# Features für das Training
X = df.dropna().copy() # nur mit Beispielen (Zeilen) trainieren, für die alle Features einen Wert haben
X.drop(columns='date', inplace=True) # Datum nutzen wir nicht als Feature
X['Country/Region'] = X['Country/Region'].cat.codes # RandomForest kann nur numerische Features handhaben => Kategorien konvertieren
X = X.astype('float32') # der Random Forest von RAPIDS mag nur Floats
    

# Trainieren
models = [] # Liste mit Modellen je Vorhersagehorizont
for h in range(1, horizon+1):
    y = X.groupby('Country/Region')['value_daily'].shift(-h) # Zielvariable aus der verschobenen Zeitreihe ableiten
    
    model = RandomForestRegressor(max_depth=8, n_estimators=100) # für schnelleres Training, z.B. beim Debugging: max_depth=2, n_estimators=10
    model.fit(X[~y.isnull()], y.dropna()) # nur mit Beispielen trainieren, für die es einen Zielwert gibt
    models.append(model)

## Vorhersage mit mehreren direkten Modellen (eins je Horizont)
Die Vorhersage beginnen wir nach dem letzten Zeitpunkt, für den wir noch Daten haben.

In [ ]:
forecast_date = df['date'].max() # Vorhersagezeitpunkt (kann auch mit pd.to_datetime('2021-01-23') gesetzt werden)

# Features für die Vorhersage
Xf = df[df['date']==forecast_date].copy()
Xf.drop(columns='date', inplace=True) # Datum nutzen wir nicht als Feature
Xf['Country/Region'] = Xf['Country/Region'].cat.codes # Kategorien in numerischen Wert wandeln
Xf = Xf.astype('float32') # der Random Forest von RAPIDS mag nur Floats

result = pd.DataFrame() # wird alle Vorhersagen mit Country/Region, Datum und Wert enthalten
for h in range(0, horizon):
    prediction = models[h].predict(Xf)
    tmp = Xf[['Country/Region']].assign(forecast_daily = prediction)
    tmp['date'] = forecast_date + pd.DateOffset(h+1)
    result = result.append(tmp)
    
result['Country/Region'] = df['Country/Region'].cat.categories[result['Country/Region'].astype('int')] # Werte der Kategorien zurückkonvertieren
result = result.append(df.loc[df['date'] <= forecast_date][['Country/Region', 'date', 'value_daily']]) # Ist-Werte zum Ergebnis hinzufügen

Ergebnis der Vorhersage plotten...

In [ ]:
fig = px.line(result,
              x='date', y=['forecast_daily', 'value_daily'], facet_row='Country/Region',
              title='Titel', labels=dict(date='Datum', value=''))
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1])) # 'Italy' anstatt 'Country/Region=Italy' für die Beschriftung der Subplots
fig.show()

## Einfluss der Features mit Shapley-Werten erklären

Mit Hilfe von Shapley-Werten lässt sich der Einfluss von Feature-Werte auf die Vorhersagen eines Modells erklären. Um Zeit zu sparen, berechnen wir die Shapley-Werte nur für eine Stichprobe. Im `shap.summary_plot` wird je Feature (y-Achse) der Einfluss des Features auf die Vorhersage (x-Achse) in Abhängigkeit des Wert des Features (Farbe) gezeigt.

In [ ]:
%%time
import shap

from cuml.experimental.explainer import PermutationExplainer as cuPE

cu_explainer = cuPE(model=models[horizon-1].predict, data=X) # ein wenig hacky: horizon sollte 7 sein, in X sollten die Trainingsdaten vom letzten Modell (7 Tage in die Zukunft) sein

X_train_sample = X.sample(100) # nur 100 Beispiele für die Visualisierung nutzen

shap_values = cu_explainer.shap_values(X_train_sample)
shap.summary_plot(shap_values, X_train_sample)

Der `shap_force_plot` zeigt für eine konkrete Vorhersage die Wirkung der einzelnen Features auf die Vorhersage.

In [ ]:
example_index = 42 # Index für ein konkretes Beispiel
shap.initjs()
shap.force_plot(cu_explainer.expected_value, shap_values[example_index], X_train_sample.values[example_index], feature_names=X_train_sample.columns.values)

# Übung

Jetzt bist Du dran. Die Übungen wirst Du nicht mit den COVID-19-Zahlen machen, sondern mit den Daten aus dem fünften M-Wettbewerb zur Vorhersage von Zeitreihen.

# Verkaufszahlen einlesen

Die Daten für unseren Workshop kommen aus dem fünften M-Wettbewerb zur Vorhersage von Zeitreihen und sind Verkaufszahlen aus 10 verschiedenen Walmarts in den USA. Für diesen Workshop haben wir die Verkaufszahlen auf eine Auswahl von Produkten reduziert (die 15 meistverkauften Produkte je Abteilung). Wir nutzen für dieses Notebook nur Daten ab dem Jahr 2015.

In [ ]:
df = pd.read_parquet('../input/m5-forecasting-parquet-and-aggregations/daily_sales_items_top105.parquet')
df = df.loc[df.date>='2015-01-01']
df['weekday'] = df['date'].dt.weekday
df.drop(columns=['d'], inplace=True)
id_columns = [c for c in df.columns.values if 'id' in c]
for c in id_columns:
    df[c] = df[c].astype('category')
df.sample(5)

# 1. Aufgabe - direkte ML-Modelle

Erstelle Modelle für die Vorhersage der nächsten 7 Tagen. Erstelle dafür für jeden Vorhersageschritt (morgen, übermorgen, ...) ein eigenes Modell. Benutze einen gleitenden Durchschnitt über die letzten 7 Tagen und erstelle ein Trend-Feature wie im COVID-19 Beispiel ebenfalls mit einer Differenz von 7 Tagen. Nutze den Wochentag als Feature aber nicht das Datum. Plotte die Vorhersagen je Kategorie.

## Feature Engineering

In [ ]:
# die beiden Features value_trend und value_trend_slope erstellen, die Spalte id eignet sich für die Gruppierung

## Training (7 direkte Modelle)

In [ ]:
# dept_id, cat_id, usw. müssen kodiert werden
# Training findet je Vorhersagehorizont statt

## Vorhersage (7 direkte Modelle)

In [ ]:
# Als Vorhersagedatum das letzte Datum aus df nutzen
# alle Vorhersagen in ein Ergbnis-Dataframe (result) schreiben

## Plot der Vorhersagen je Kategorie

In [ ]:
# Zuerst den Ergebnis-Dataframe je Kategorie und Datum aggregieren, dann plotten (facet_row='cat_id')

# 2. Aufgabe - Shapley-Werte [*optional*]

Plotte die Shapley-Werte für das letzte Modell mit dem Vorhersagehorizont von 7 Tagen.

In [ ]:
# ggf. zuerst nur mit einer Stichprobe von 10 

# 3. Aufgabe - Direktes Modell mit variabler Schrittweite [*optional*]

Erstelle ein direktes Modell, dass alle sieben nächsten Tag vorhersagen kann (mit einem Feature `step_length`). Vergleiche dann die Vorhersage mit der Vorhersage aus den direkten Modellen mit statischem Vorhersagehorizont.

In [ ]:
# 1. Die Trainingsdaten erstellen: je Vorhersagehorizont die Daten jeweils kopieren, step_length setzen und die Zielvariable mit shift berechnen
# alles in einen DataFrame für X, am Ende die Zielwerte nach y extrahieren 
# 2. Modell auf X und y trainieren
# 3. Xf für die Vorhersage erstellen.
# 4. Ergebnis der Vorhersage in einen eigenen DataFrame speichern (nicht das aus der ersten Aufgabe überschreiben)
# 5. Für den Plot zum Vergleich die beiden Ergebnis-DataFrames mergen
# Hinweise:
#    - ein DataFrame <mydf> kann z.B. <times>-mal kopiert werden: mydf.loc[mydf.index.repeat(times)]
#    - eine Sequenz von <min_value> bis <max_value> kann <times>-mal wiederholt werden: list(range(min_value, max_value+1))*times

# Musterlösung: 1. Aufgabe - direkte Modelle

## Feature Engineering

In [ ]:
df.sort_values(['date'], inplace=True)
df['value_trend'] = df.groupby(['id'])['value'].rolling(window=7).mean().reset_index(level=0, drop=True)
df['value_trend_slope'] = df.groupby(['id'])['value_trend'].diff(7)
df.sample(5)

## Training (7 direkte Modelle)

In [ ]:
%%time

from cuml.ensemble import RandomForestRegressor

horizon = 7

# Features für das Training
X = df.dropna().copy() # nur mit Beispielen (Zeilen) trainieren, für die alle Features einen Wert haben
X.drop(columns=['id', 'date'], inplace=True) # ID, Jahr und Monat nutzen wir nicht als Feature
X['dept_id'] = X['dept_id'].cat.codes # RandomForest kann nur numerische Features handhaben => Kategorien konvertieren
X['cat_id'] = X['cat_id'].cat.codes # RandomForest kann nur numerische Features handhaben => Kategorien konvertieren
X['store_id'] = X['store_id'].cat.codes # RandomForest kann nur numerische Features handhaben => Kategorien konvertieren
X['state_id'] = X['state_id'].cat.codes # RandomForest kann nur numerische Features handhaben => Kategorien konvertieren
X['item_id'] = X['item_id'].cat.codes # RandomForest kann nur numerische Features handhaben => Kategorien konvertieren
X = X.astype('float32') # der Random Forest von RAPIDS mag nur Floats

# Trainieren
models = [] # Liste mit Modellen je Vorhersagehorizont
for h in range(1, horizon+1):
    y = X.groupby(['store_id', 'dept_id', 'item_id'])['value'].shift(-h) # Zielvariable aus der verschobenen Zeitreihe ableiten
    
    model = RandomForestRegressor(max_depth=8, n_estimators=100)
    model.fit(X[~y.isnull()], y.dropna()) # nur mit Beispielen trainieren, für die es einen Zielwert gibt
    models.append(model)

## Vorhersage (7 direkte Modelle)

In [ ]:
forecast_date = df['date'].max()

# Features für die Vorhersage
Xf = df[df['date']==forecast_date].copy()
Xf.drop(columns=['id', 'date'], inplace=True) # ID, Jahr und Monat nutzen wir nicht als Feature

Xf['dept_id'] = Xf['dept_id'].cat.codes # RandomForest kann nur numerische Features handhaben => Kategorien konvertieren
Xf['cat_id'] = Xf['cat_id'].cat.codes # RandomForest kann nur numerische Features handhaben => Kategorien konvertieren
Xf['store_id'] = Xf['store_id'].cat.codes # RandomForest kann nur numerische Features handhaben => Kategorien konvertieren
Xf['state_id'] = Xf['state_id'].cat.codes # RandomForest kann nur numerische Features handhaben => Kategorien konvertieren
Xf['item_id'] = Xf['item_id'].cat.codes # RandomForest kann nur numerische Features handhaben => Kategorien konvertieren
Xf = Xf.astype('float32')

result = pd.DataFrame() # wird alle Vorhersagen mit Country/Region, Datum und Wert enthalten
for h in range(0, horizon):
    prediction = models[h].predict(Xf)
    tmp = Xf[['dept_id', 'cat_id', 'store_id', 'state_id', 'item_id']].assign(forecast_value = prediction)
    tmp['date'] = forecast_date + pd.DateOffset(days=h+1)
    result = result.append(tmp)
    
result['dept_id'] = df['dept_id'].cat.categories[result['dept_id'].astype('int')] # Werte der Kategorien zurückkonvertieren
result['cat_id'] = df['cat_id'].cat.categories[result['cat_id'].astype('int')] # Werte der Kategorien zurückkonvertieren
result['store_id'] = df['store_id'].cat.categories[result['store_id'].astype('int')] # Werte der Kategorien zurückkonvertieren
result['state_id'] = df['state_id'].cat.categories[result['state_id'].astype('int')] # Werte der Kategorien zurückkonvertieren
result['item_id'] = df['item_id'].cat.categories[result['item_id'].astype('int')] # Werte der Kategorien zurückkonvertieren
result = result.append(df.loc[df['date'] <= forecast_date][['dept_id', 'cat_id', 'store_id', 'state_id', 'item_id', 'date', 'value']]) # Ist-Werte zum Ergebnis hinzufügen

## Plot der Vorhersagen je Kategorie

In [ ]:
facet_column = 'cat_id' # probehalber auch mal mit state_id ausprobieren

fig = px.line(result.
                  groupby([facet_column, 'date']).
                  agg({'value': sum, 'forecast_value': sum}).
                  reset_index(),
              x='date', y=['value', 'forecast_value'], facet_row=facet_column,
              title='Titel')
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.update_yaxes(matches=None)
fig.show()

# Musterlösung: 2. Aufgabe - Shapley-Werte [*optional*]

In [ ]:
%%time
import shap

from cuml.experimental.explainer import PermutationExplainer as cuPE

cu_explainer = cuPE(model=models[horizon-1].predict, data=X) # ein wenig hacky: horizon sollte 7 sein, in X sollten die Trainingsdaten vom letzten Modell (7 Tage in die Zukunft) sein

X_train_sample = X.sample(100) # nur 100 Beispiele für die Visualisierung nutzen

shap_values = cu_explainer.shap_values(X_train_sample)
shap.summary_plot(shap_values, X_train_sample)

# 3. Aufgabe - Direktes Modell mit variabler Schrittweite [*optional*]

In [ ]:
X = pd.DataFrame()
horizon = 7
for step_length in range(1, horizon+1):
    tmp = df.copy()
    tmp['step_length'] = step_length
    tmp['target_value'] = tmp.groupby(['store_id', 'dept_id', 'item_id'])['value'].shift(-step_length)
    X = tmp.append(X)

X = X.dropna().copy() # nur mit Beispielen (Zeilen) trainieren, für die alle Features einen Wert haben
X.drop(columns=['id', 'date'], inplace=True) # Jahr und Monat nutzen wir nicht als Feature
X['dept_id'] = X['dept_id'].cat.codes # RandomForest kann nur numerische Features handhaben => Kategorien konvertieren
X['cat_id'] = X['cat_id'].cat.codes # RandomForest kann nur numerische Features handhaben => Kategorien konvertieren
X['store_id'] = X['store_id'].cat.codes # RandomForest kann nur numerische Features handhaben => Kategorien konvertieren
X['state_id'] = X['state_id'].cat.codes # RandomForest kann nur numerische Features handhaben => Kategorien konvertieren
X['item_id'] = X['item_id'].cat.codes # RandomForest kann nur numerische Features handhaben => Kategorien konvertieren

X = X.astype('float32') # der Random Forest von RAPIDS mag nur Floats

y = X['target_value']

X.drop(columns='target_value', inplace=True)
X.sample(5)

In [ ]:
model = RandomForestRegressor(max_depth=8, n_estimators=100)
model.fit(X[~y.isnull()], y.dropna()) # nur mit Beispielen trainieren, für die es einen Zielwert gibt

In [ ]:
forecast_date = df['date'].max()

Xf = df[df['date']==forecast_date].copy()
Xf.drop(columns=['id', 'date'], inplace=True) # ID, Jahr und Monat nutzen wir nicht als Feature

Xf['dept_id'] = Xf['dept_id'].cat.codes # RandomForest kann nur numerische Features handhaben => Kategorien konvertieren
Xf['cat_id'] = Xf['cat_id'].cat.codes # RandomForest kann nur numerische Features handhaben => Kategorien konvertieren
Xf['store_id'] = Xf['store_id'].cat.codes # RandomForest kann nur numerische Features handhaben => Kategorien konvertieren
Xf['state_id'] = Xf['state_id'].cat.codes # RandomForest kann nur numerische Features handhaben => Kategorien konvertieren
Xf['item_id'] = Xf['item_id'].cat.codes # RandomForest kann nur numerische Features handhaben => Kategorien konvertieren

row_count = len(Xf)

Xf = Xf.loc[Xf.index.repeat(horizon)]
Xf['step_length'] = list(range(1, horizon+1))*row_count

Xf = Xf.astype('float32')

prediction = model.predict(Xf)

result2 = Xf[['dept_id', 'cat_id', 'store_id', 'state_id', 'item_id', 'step_length']].copy()
result2['date'] = forecast_date + result2['step_length'].astype('timedelta64[D]')
result2['forecast_value_vsl'] = prediction

result2['dept_id'] = df['dept_id'].cat.categories[result2['dept_id'].astype('int')] # Werte der Kategorien zurückkonvertieren
result2['cat_id'] = df['cat_id'].cat.categories[result2['cat_id'].astype('int')] # Werte der Kategorien zurückkonvertieren
result2['store_id'] = df['store_id'].cat.categories[result2['store_id'].astype('int')] # Werte der Kategorien zurückkonvertieren
result2['state_id'] = df['state_id'].cat.categories[result2['state_id'].astype('int')] # Werte der Kategorien zurückkonvertieren
result2['item_id'] = df['item_id'].cat.categories[result2['item_id'].astype('int')] # Werte der Kategorien zurückkonvertieren

result_comparison = result.merge(result2, how='left', on=['dept_id', 'cat_id', 'store_id', 'state_id', 'item_id', 'date'])

In [ ]:
facet_column = 'cat_id' # probehalber auch mal mit state_id ausprobieren

fig = px.line(result_comparison.
                  groupby([facet_column, 'date']).
                  agg({'value': sum, 'forecast_value': sum, 'forecast_value_vsl': sum}).
              reset_index(),
              x='date', y=['value', 'forecast_value', 'forecast_value_vsl'], facet_row=facet_column,
              title='Titel')
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.update_yaxes(matches=None)
fig.show()